In [ ]:
#| hide
from calibrate_simulation.calibrate_simulation import *

# calibrate-simulation

> A Metamodel-Based General-purpose Calibration Tool for Simulation Models

This package is implementing the metamodel optimization method described in [1]. The aim of this repository is to provide a base sample implementation that can be used to to train a metamodel based on a given sample input – output result set then create an optimization model based on the trained metamodel and finally get candidate input parameters that will generate any given desired output. 

A simple use case for the module consists of 3 steps: 
1.	initialize the module object, 
2.	train the metamodel by supplying training and validation datasets
3.	 Use optimization model to create candidate input parameters given lower and upper bounds and target output 


## Install

```sh
pip install calibrate_simulation
```

## How to use

When creating the optimizer object and initializing the module the user must choose between implemented two metamodel types and two MIP solvers. The metamodel types are:

1. Random Forest based metamodel. (not yet implemented)
2. Artificial neural network based metamodel. 

The available MIP solvers for the optimization step are:

1. GUROBI solver (To use the GUROBI solver the user hast to provide the necessary licenses).
2. Google OR Tools based SCIP solver.

## Minimal Sample Code

In [ ]:
#| eval: false
import numpy as np

# Random data for example purposes
X_training = np.random.randn(20000,6)
X_validation = np.random.randn(5000,6)
X_test = np.random.randn(4000,6)

Y_training = np.random.randn(20000,9)
Y_validation = np.random.randn(5000,9)
Y_test = np.random.randn(4000,9)

# Initialize the Calibration object 
optimizer = CalibrateSimulation(OptimizerType.OR_TOOLS)

# Train the metamodel 
optimizer.train_model(X_training, Y_training, X_validation, Y_validation)

# 9 sample target values for output
target_sample = np.array([0.72864795, 0.72025004, 0.66572048, 0.68154454, 0.65445883, 0.57947686, 0.60197869, 0.53777778, 0.56603774])

#lower and upper bounds for input values. These values will be used as constraints for the optimization 
lower_bound = [1, 0.5, 5, 1, 0.4, 390]
upper_bound = [2.5, 1, 10, 5, 0.8,760]

# Use selected solver to create and solve the optimization model for given sample target values
result = optimizer.solve_optimization(target_sample,lower_bound,upper_bound)

## FULL Working Sample

### 1. Prepare training datasets

In [ ]:
#| eval: false
import numpy as np
import pandas as pd

M = pd.read_csv('testing/data/output-seed_1337-30000.csv')
#NORMALIZATION
M.drop(M.columns[[8,12,13,16,17,18,21]], axis = 1, inplace= True)
for i in range(6,15):
    M.iloc[:,i] = (M.iloc[:,i] - M.iloc[:,i].min())/(M.iloc[:,i].max() - M.iloc[:,i].min())
    target_sample = M.sample(100)
M = M.drop(target_sample.index)
M.reset_index(inplace=True)
M.drop(M.columns[0], axis = 1, inplace=True)
M.to_csv('testing/data/OutputMinMaxScaledResults30000.csv')
target_sample.to_csv('testing/data/TargetSample.csv')

M_train = M.sample(frac=0.7,  random_state=1337)
M_rest = M.drop(M_train.index)
M_validation = M_rest.sample(frac=0.5,  random_state=1337)
M_test = M_rest.drop(M_validation.index)

X_training = np.array(M_train.iloc[:,:6])
X_validation = np.array(M_validation.iloc[:,:6])
X_test = np.array(M_test.iloc[:,:6])

Y_training = np.array(M_train.iloc[:,6:])
Y_validation = np.array(M_validation.iloc[:,6:])
Y_test = np.array(M_test.iloc[:,6:])

### 2. Prepare target value datasets

In [ ]:
#| eval: false
Target_Sample = pd.read_csv('testing/data/TargetSample.csv', index_col=0).iloc[:,[6,7,8,9,10,11,12,13,14]]
Target_Sample.reset_index(inplace=True)
Target_Sample.drop(Target_Sample.columns[0], axis = 1, inplace=True)
target_val = np.array(Target_Sample[0:1]).tolist()[0]

### 3. Train NN model

In [ ]:
#|output: false
#| eval: false

optimizer = CalibrateSimulation(OptimizerType.OR_TOOLS)
optimizer.train_model(X_training, Y_training, X_validation, Y_validation)


### 4. Generate calibration inputs using optimization

In [ ]:
#|output: false
#| eval: false

sample_arrays = np.array(Target_Sample)
lower_bound = [1, 0.5, 5, 1, 0.4, 390]
upper_bound = [2.5, 1, 10, 5, 0.8,760]

results_param = {}
results_output = {}
for i in range(100):
    result = optimizer.solve_optimization(sample_arrays[i],lower_bound,upper_bound)
    results_param[i] = result[0]
    results_output[i] = result[1]
    print(f'{i} done')


### 5. Print results

In [ ]:
#| eval: false

print(results_param)
print(results_output)